<a href="https://colab.research.google.com/github/tuhin-datascience/SQL_in_Google_Colab/blob/dev-branch/SQL_Windows_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SQL Windows Functions**
Magic commands in IPython are a useful set of commands that help solve common problems while working with data. One such command is the SQL magic command that allows writing SQL queries within a notebook.

First, we need to install the ipython-sql extension. This can be done directly in a Colab cell:

In [1]:
# Install ipython-sql
!pip install ipython-sql --quiet


Next, load the SQL extension and create a SQLite database:

In [2]:
# Load the SQL extension
%load_ext sql

# Create a SQLite database
%sql sqlite://


### Tables Creation:

Here, we have created two tables, products and product_group, product table contains infomation regarding the products and the product_group table contains the informstion regrding the product group.


In [3]:
%%sql
CREATE TABLE IF NOT EXISTS products(
  product_id interger primary key,
  product_name varchar(255) not null,
  price decimal(10, 2) not null,
  group_id integer not null,
  foreign key (group_id) references product_group(group_id)
);

 * sqlite://
Done.


[]

In [4]:
%%sql

CREATE TABLE IF NOT EXISTS product_group(
group_id INTEGER PRIMARY KEY,
group_name VARCHAR(255) NOT NULL

);

 * sqlite://
Done.


[]

In [5]:
%%sql

INSERT INTO product_group(group_id, group_name) VALUES (1,'Smartphone');
INSERT INTO product_group(group_id, group_name) VALUES (2, 'Laptop');
INSERT INTO product_group(group_id, group_name) VALUES (3, 'Tablet');

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [6]:
%%sql
INSERT INTO products (product_name, group_id,price)
VALUES
	('Microsoft Lumia', 1, 200),
	('HTC One', 1, 400),
	('Nexus', 1, 500),
	('iPhone', 1, 900),
	('HP Elite', 2, 1200),
	('Lenovo Thinkpad', 2, 700),
	('Sony VAIO', 2, 700),
	('Dell Vostro', 2, 800),
	('iPad', 3, 700),
	('Kindle Fire', 3, 150),
	('Samsung Galaxy Tab', 3, 200);

 * sqlite://
11 rows affected.


[]

### **Introduction to PostgreSQL window functions**
The easiest way to understand the window functions is to start by reviewing the aggregate functions. An aggregate function aggregates data from a set of rows into a single row.

The following example uses the AVG() aggregate function to calculate the average price of all products in the products table.

In [7]:
%%sql

SELECT AVG(price) FROM products;

 * sqlite://
Done.


AVG(price)
586.3636363636364


To apply the aggregate function to subsets of rows, you use the GROUP BY clause. The following example returns the average price for every product group.

In [9]:
%%sql

select group_name, avg(price) from products
inner join product_group on products.group_id = product_group.group_id
group by group_name;

 * sqlite://
Done.


group_name,avg(price)
Laptop,850.0
Smartphone,500.0
Tablet,350.0


As you see clearly from the output, the AVG() function reduces the number of rows returned by the queries in both examples.

Similar to an aggregate function, a window function operates on a set of rows. However, it does not reduce the number of rows returned by the query.

The term window describes the set of rows on which the window function operates. A window function returns values from the rows in a window.

For instance, the following query returns the product name, the price, product group name, along with the average prices of each product group.

In [10]:
%%sql

select product_name, price, group_name, avg(price)
over (partition by group_name) as avg_price
from products
inner join product_group
on products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,avg_price
HP Elite,1200,Laptop,850.0
Lenovo Thinkpad,700,Laptop,850.0
Sony VAIO,700,Laptop,850.0
Dell Vostro,800,Laptop,850.0
Microsoft Lumia,200,Smartphone,500.0
HTC One,400,Smartphone,500.0
Nexus,500,Smartphone,500.0
iPhone,900,Smartphone,500.0
iPad,700,Tablet,350.0
Kindle Fire,150,Tablet,350.0


In this query, the AVG() function works as a window function that operates on a set of rows specified by the OVER clause. Each set of rows is called a window.

The new syntax for this query is the OVER clause:

`AVG(price) OVER (PARTITION BY group_name)`

In this syntax, the PARTITION BY distributes the rows of the result set into groups and the AVG() function is applied to each group to return the average price for each.

Note that a window function always performs the calculation on the result set after the JOIN, WHERE, GROUP BY and HAVING clause and before the final ORDER BY clause in the evaluation order.

### **PARTITION BY clause**
The PARTITION BY clause divides rows into multiple groups or partitions to which the window function is applied. Like the example above, we used the product group to divide the products into groups (or partitions).

The PARTITION BY clause is optional. If you skip the PARTITION BY clause, the window function will treat the whole result set as a single partition.

### **ORDER BY clause**
The ORDER BY clause specifies the order of rows in each partition to which the window function is applied.

The ORDER BY clause uses the NULLS FIRST or NULLS LAST option to specify whether nullable values should be first or last in the result set. The default is NULLS LAST option.

### **SQL window function List**
The following table lists all window functions provided by PostgreSQL. Note that some aggregate functions such as AVG(), MIN(), MAX(), SUM(), and COUNT() can be also used as window functions.

### **Name	Description**
CUME_DIST	Return the relative rank of the current row.
DENSE_RANK	Rank the current row within its partition without gaps.
FIRST_VALUE	Return a value evaluated against the first row within its partition.
LAG	Return a value evaluated at the row that is at a specified physical offset row before the current row within the partition.
LAST_VALUE	Return a value evaluated against the last row within its partition.
LEAD	Return a value evaluated at the row that is offset rows after the current row within the partition.
NTILE	Divide rows in a partition as equally as possible and assign each row an integer starting from 1 to the argument value.
NTH_VALUE	Return a value evaluated against the nth row in an ordered partition.
PERCENT_RANK	Return the relative rank of the current row (rank-1) / (total rows – 1)
RANK	Rank the current row within its partition with gaps.
ROW_NUMBER	Number the current row within its partition starting from 1.

### **The ROW_NUMBER(), RANK(), and DENSE_RANK() functions**
The ROW_NUMBER(), RANK(), and DENSE_RANK() functions assign an integer to each row based on its order in its result set.

The ROW_NUMBER() function assigns a sequential number to each row in each partition. See the following query:

In [13]:
%%sql

select product_name, price, group_name,
row_number() over(order by price) as [rn]
from products
inner join product_group
on products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,rn
Kindle Fire,150,Tablet,1
Microsoft Lumia,200,Smartphone,2
Samsung Galaxy Tab,200,Tablet,3
HTC One,400,Smartphone,4
Nexus,500,Smartphone,5
Lenovo Thinkpad,700,Laptop,6
Sony VAIO,700,Laptop,7
iPad,700,Tablet,8
Dell Vostro,800,Laptop,9
iPhone,900,Smartphone,10


In [15]:
%%sql

select product_name, price, group_name,
row_number() over(partition by group_name order by price) as [rn]
from products
inner join product_group
on products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,rn
Lenovo Thinkpad,700,Laptop,1
Sony VAIO,700,Laptop,2
Dell Vostro,800,Laptop,3
HP Elite,1200,Laptop,4
Microsoft Lumia,200,Smartphone,1
HTC One,400,Smartphone,2
Nexus,500,Smartphone,3
iPhone,900,Smartphone,4
Kindle Fire,150,Tablet,1
Samsung Galaxy Tab,200,Tablet,2


The RANK() function assigns ranking within an ordered partition. If rows have the same values, the  RANK() function assigns the same rank, with the next ranking(s) skipped.

See the following query:

In [16]:
%%sql

select product_name, price, group_name,
rank() over(partition by group_name order by price) as [rn]
from products
inner join product_group
on products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,rn
Lenovo Thinkpad,700,Laptop,1
Sony VAIO,700,Laptop,1
Dell Vostro,800,Laptop,3
HP Elite,1200,Laptop,4
Microsoft Lumia,200,Smartphone,1
HTC One,400,Smartphone,2
Nexus,500,Smartphone,3
iPhone,900,Smartphone,4
Kindle Fire,150,Tablet,1
Samsung Galaxy Tab,200,Tablet,2


Similar to the RANK() function, the DENSE_RANK() function assigns a rank to each row within an ordered partition, but the ranks have no gap. In other words, the same ranks are assigned to multiple rows and no ranks are skipped.

In [17]:
%%sql

select product_name, price, group_name,
dense_rank() over(partition by group_name order by price) as [rn]
from products
inner join product_group
on products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,rn
Lenovo Thinkpad,700,Laptop,1
Sony VAIO,700,Laptop,1
Dell Vostro,800,Laptop,2
HP Elite,1200,Laptop,3
Microsoft Lumia,200,Smartphone,1
HTC One,400,Smartphone,2
Nexus,500,Smartphone,3
iPhone,900,Smartphone,4
Kindle Fire,150,Tablet,1
Samsung Galaxy Tab,200,Tablet,2


### **The FIRST_VALUE and LAST_VALUE functions**
The FIRST_VALUE() function returns a value evaluated against the first row within its partition, whereas the LAST_VALUE() function returns a value evaluated against the last row in its partition.

The following statement uses the FIRST_VALUE() to return the lowest price for every product group.

In [21]:
%%sql

select product_name, price, group_name,
FIRST_VALUE(price) over(partition by group_name order by price) as [fv]
from products
inner join product_group
ON products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,fv
Lenovo Thinkpad,700,Laptop,700
Sony VAIO,700,Laptop,700
Dell Vostro,800,Laptop,700
HP Elite,1200,Laptop,700
Microsoft Lumia,200,Smartphone,200
HTC One,400,Smartphone,200
Nexus,500,Smartphone,200
iPhone,900,Smartphone,200
Kindle Fire,150,Tablet,150
Samsung Galaxy Tab,200,Tablet,150


In [23]:
%%sql

select product_name, price, group_name,
LAST_VALUE(price) over(partition by group_name order by price
range between unbounded preceding and unbounded following) as [lv]
from products
inner join product_group
ON products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,lv
Lenovo Thinkpad,700,Laptop,1200
Sony VAIO,700,Laptop,1200
Dell Vostro,800,Laptop,1200
HP Elite,1200,Laptop,1200
Microsoft Lumia,200,Smartphone,900
HTC One,400,Smartphone,900
Nexus,500,Smartphone,900
iPhone,900,Smartphone,900
Kindle Fire,150,Tablet,700
Samsung Galaxy Tab,200,Tablet,700


Notice that we added the frame clause `RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` because by default the frame clause is `RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`.

### **The LAG and LEAD functions**
The `LAG()` function has the ability to access data from the previous row, while the `LEAD()` function can access data from the next row.

Both `LAG()` and `LEAD()` functions have the same syntax as follows:


```
LAG  (expression [,offset] [,default]) over_clause;
LEAD (expression [,offset] [,default]) over_clause;

```
The following statement uses the LAG() function to return the prices from the previous row and calculates the difference between the price of the current row and the previous row.


In [25]:
%%sql

select product_name, price, group_name,
lag(price, 1, 0) over(partition by group_name order by price) as [prev_price],
price - lag(price, 1, 0) over(partition by group_name order by price) as [price_diff]
from products
inner join product_group
on products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,prev_price,price_diff
Lenovo Thinkpad,700,Laptop,0,700
Sony VAIO,700,Laptop,700,0
Dell Vostro,800,Laptop,700,100
HP Elite,1200,Laptop,800,400
Microsoft Lumia,200,Smartphone,0,200
HTC One,400,Smartphone,200,200
Nexus,500,Smartphone,400,100
iPhone,900,Smartphone,500,400
Kindle Fire,150,Tablet,0,150
Samsung Galaxy Tab,200,Tablet,150,50


The following statement uses the `LEAD()` function to return the prices from the next row and calculates the difference between the price of the current row and the next row.

In [26]:
%%sql

select product_name, price, group_name,
lead(price, 1, 0) over(partition by group_name order by price) as [next_price],
price - lead(price, 1, 0) over(partition by group_name order by price) as [price_diff]
from products
inner join product_group
on products.group_id = product_group.group_id;

 * sqlite://
Done.


product_name,price,group_name,next_price,price_diff
Lenovo Thinkpad,700,Laptop,700,0
Sony VAIO,700,Laptop,800,-100
Dell Vostro,800,Laptop,1200,-400
HP Elite,1200,Laptop,0,1200
Microsoft Lumia,200,Smartphone,400,-200
HTC One,400,Smartphone,500,-100
Nexus,500,Smartphone,900,-400
iPhone,900,Smartphone,0,900
Kindle Fire,150,Tablet,200,-50
Samsung Galaxy Tab,200,Tablet,700,-500
